In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from geopy.geocoders import Nominatim
! pip install folium
import folium

print('Packages installed')

     |████████████████████████████████| 92kB 1.0MB/s eta 0:00:01
Packages installed


In [7]:
# Real estate data
sheet_url_RD = 'https://docs.google.com/spreadsheets/d/1CiUj-cWcO1D7KN_qfs6jArjfHf9njpUGlrc3Lk2GxO0/edit#gid=915412023'
csv_export_url_RD = sheet_url_RD.replace('/edit#gid=', '/export?format=csv&gid=')
aruodas_df = pd.read_csv(csv_export_url_RD)
aruodas_df.rename(columns={'Adresas': 'Adress',
                     'Rajonas':'Neighbourhood',
                     'Plotas':'Area',
                     'Kaina':'Price',
                     'Metai': 'Year',
                     'Šildymas': 'Heating',
                     'Kambarių sk': 'Rooms',
                     'Pastato tipas':'Type'}, inplace=True)
neigh_list = aruodas_df['Neighbourhood'].unique()


In [13]:
# DataFrame for housing prices based on current market in aruodas.lt advertisments.
aruodas_df.head()

,Adress,Neighbourhood,Rooms,Area,Price,Heating,Type,Year
0,Elbingo g.,Pilaitė,1,64.0,87000,Centriniskolektorinis,Mūrinis,2018
1,Elbingo g.,Pilaitė,3,55.0,76000,Centriniskolektorinis,Mūrinis,2018
2,Elbingo g.,Pilaitė,2,48.0,69000,Centriniskolektorinis,Mūrinis,2018
3,Elbingo g.,Pilaitė,2,48.0,86000,Centriniskolektorinis,Mūrinis,2018
4,Žaliųjų Ežerų g.,Santariškės,2,51.5,88000,Centrinis,Mūrinis,1982statyba.2017renovacija


In [14]:
#Size of the dataframe
aruodas_df.shape

(3684, 8)

In [4]:
# Companies in Vilnius data
sheet_url_ED = 'https://docs.google.com/spreadsheets/d/1JKj4-dYQqW6gB_Xr3inEZfr88YogIDVTGbezOgbJxIc/edit#gid=1786717215'
csv_export_url_ED = sheet_url_ED.replace('/edit#gid=', '/export?format=csv&gid=')
enterprise_df = pd.read_csv(csv_export_url_ED)
enterprise_df.head()

,Draudėjo kodas (code),Juridinių asmenų registro kodas (jarCode),Pavadinimas (name),Savivaldybė kurioje registruota(municipality),Ekonominės veiklos rūšies kodas(ecoActCode),Ekonominės veiklos rūšies pavadinimas(ecoActName),Mėnuo (month),Vidutinis darbo užmokestis (avgWage),Apdraustųjų skaičius (numInsured),Vidutinis darbo užmokestis II (avgWage2),Apdraustųjų skaičius II (numInsured2),Valstybinio socialinio draudimo įmoka (tax)
0,42399,NaN,LATVIJOS RESPUBLIKOS AMBASADA,Vilniaus m. sav.,NaN,NaN,201901,NaN,3,NaN,0,NaN
1,44717,NaN,APAŠTALINE NUNCIATURA,Vilniaus m. sav.,NaN,NaN,201901,938.37,6,NaN,0,1233.92
2,45838,NaN,GOETHE-INSTITUT,Vilniaus m. sav.,NaN,NaN,201901,1712.98,8,NaN,0,3075.15
3,50764,NaN,BRITU TARYBA,Vilniaus m. sav.,NaN,NaN,201901,2266.82,5,NaN,0,2101.23
4,51006,NaN,LENKIJOS RESPUBLIKOS AMBASADA,Vilniaus m. sav.,NaN,NaN,201901,1278.02,22,NaN,0,6355.74


In [19]:
enterprise_name = enterprise_df['Pavadinimas (name)'].to_list()
fix_name = []
for i in enterprise_name:
    ab = 'AKCINE'
    uab = 'UŽDAROJI'
    vsi = 'VIEŠOJI'
    vi = 'VALSTYBES'
    if uab in i:
        name = i.replace('UŽDAROJI','').replace('AKCINE','').replace('BENDROVE','UAB')
        if name[0] == ' ':
            new_name = name[1:]
            if new_name[0] == ' ':
                new_name_1 = new_name[1:]
                fix_name.append(new_name_1)
            else:
                fix_name.append(new_name)
        else:
            fix_name.append(name)
    elif ab in i:
        name = i.replace('AKCINE','').replace('BENDROVE','AB')
        if name[0] == ' ':
            new_name = name[1:]
            fix_name.append(new_name)
        else:
            fix_name.append(name)
    elif vsi in i:
        name = i.replace('VIEŠOJI','').replace('ISTAIGA','Všį')
        if name[0] == ' ':
            new_name = name[1:]
            fix_name.append(new_name)
        else:
            fix_name.append(name)
    elif vi in i:
        name = i.replace('VALSTYBES','').replace('IMONE','Vį')
        if name[0] == ' ':
            new_name = name[1:]
            fix_name.append(new_name)
        else:
            fix_name.append(name)
    else:
        fix_name.append(i)
len(fix_name)

34978

In [15]:
len(enterprise_name)

34978

In [3]:
geolocator = Nominatim()
lat_list = []
long_list = []
for i in neigh_list:
    locations = geolocator.geocode(i)
    try:
        lat_list.append(locations.latitude)
        long_list.append(locations.longitude)
    except:
        lat_list.append(None)
        long_list.append(None)
        

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [5]:
grouped_data = pd.DataFrame(data={'Neighbourhood':neigh_list, 'Latitude':lat_list, 'Longitude':long_list})
grouped_data.dropna(inplace=True)
grouped_data.sort_values(by=['Neighbourhood']).reset_index().drop(columns=['index'])
grouped_data.head()

,Neighbourhood,Latitude,Longitude
0,Pilaitė,54.705676,25.183502
1,Santariškės,54.751423,25.279425
2,Žvėrynas,54.694633,25.251033
3,Pašilaičiai,54.728487,25.228916
4,Burbiškės,54.751350,24.527134


In [48]:
data['Price/area'] = data['Price'].div(data['Area'])
mean_price = data.groupby('Neighbourhood')['Price/area'].agg({'Average price':'mean'}).reset_index()
mean_price.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


,Neighbourhood,Average price
0,Antakalnis,1960.942613
1,Antavilis,973.478085
2,Aukštieji Paneriai,923.813263
3,Avižieniai,1421.594033
4,Avižienių k.,850.000000


In [22]:
address = 'Vilnius'

geolocator = Nominatim(user_agent="vilnius_explore")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [3]:
# create map of Toronto using latitude and longitude values
map_vilnius = folium.Map(location=[latitude, longitude], zoom_start=10)

# Filter stuff from dataframe
neighbourhood = geo_data['Neighbourhood']

# add markers to map
for lat, lng, neighbourhood in zip(geo_data['Latitude'], geo_data['Longitude'], geo_data['Neighbourhood']):
    label = '{}, Vilnius'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vilnius)  

NameError: name 'latitude' is not defined

In [2]:
map_vilnius.choropleth(
    geo_data=geo_data,
    data=sf_df_grouped,
    columns=['Neighbourhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Avg apartment price in Vilnius',
    reset=True
)
map_vilnius

NameError: name 'map_vilnius' is not defined